### h2oGPTe Python Client - hatespeech detection test
- h2oGPTeに対し、Python Client (h2ogpteパッケージ)でアクセス
- LLMへの質問を実施

In [1]:
from h2ogpte import H2OGPTE

import pandas as pd
from sklearn.metrics import f1_score, accuracy_score

from tqdm import tqdm

In [2]:
f = open('tmp/h2ogpte_key_hatespeechdetect.txt', 'r')
API_KEY = f.read()
f.close()

In [3]:
client = H2OGPTE(
    address="https://playground.h2ogpte.h2o.ai",
    api_key=API_KEY)
client

Please install the correct version of H2OGPTE with `pip install h2ogpte==1.5.0-dev4`.
You can enable strict version checking by passing strict_version_check=True.


In [25]:
# LLM一覧
models = client.get_llms()
[m['base_model'] for m in models]

['h2oai/h2ogpt-4096-llama2-70b-chat',
 'NousResearch/Nous-Capybara-34B',
 'mistralai/Mistral-7B-Instruct-v0.2',
 'h2oai/h2o-danube2-1.8b-chat',
 'SeaLLMs/SeaLLM-7B-v2.5',
 'mistral-small-latest',
 'mistral-large-latest',
 'mistral-medium',
 'claude-3-sonnet-20240229',
 'claude-3-opus-20240229',
 'claude-3-haiku-20240307',
 'gemini-1.5-pro-latest',
 'gpt-3.5-turbo-0613',
 'gpt-3.5-turbo-16k-0613',
 'gpt-35-turbo-1106',
 'gpt-4-1106-preview']

In [20]:
chat_session_id = client.create_chat_session_on_default_collection()
chat_session_id

'6cef0ced-d307-4124-b264-dd85f9089859'

In [21]:
MODEL = "mistralai/Mistral-7B-Instruct-v0.2"
SYS_PROMPT = "あなたは文章がヘイトスピーチかどうかを判定するAIです。質問には必ずYes(ヘイトスピーチ)かNo(ヘイトスピーチでない)で答えます。判定の理由は述べる必要はありません。"
#SYS_PROMPT = "You are an AI assystant judging the given text is hate speech or not. Please answer Yes (hate speec) or No (not hate speech). No explanation is needed. Answer only Yes or No."

In [22]:
text = '''"稲田、今井、豊田、昭恵とあんだけ女に足引っ張られたら
女性活躍なんて言ってられんよなｗ
減らされて当然なのをわからないから女は余計にダメなんだよ。"！'''

#text = '''"ま、バイト（フリーター)でもいいんじゃないの？
#後悔しないのなら"'''
text

'"稲田、今井、豊田、昭恵とあんだけ女に足引っ張られたら\n女性活躍なんて言ってられんよなｗ\n減らされて当然なのをわからないから女は余計にダメなんだよ。"！'

In [24]:
with client.connect(chat_session_id) as session:
    answer = session.query(
        message = text,
        system_prompt=SYS_PROMPT,
        llm=MODEL,
        llm_args={"max_new_tokens":256},
        rag_config={"rag_type": "llm_only",},
    ).content
    print(answer)

Yes. この文章は、女性に対する不尊重な言葉を含み、ヘイトスピーチです。


[Session.query()](https://h2oai.github.io/h2ogpte/h2ogpte.html#h2ogpte.session.Session.query)

In [50]:
answer

'Yes. この文章は、女性に対する不尊重な言葉を含み、ヘイトスピーチです。'

In [53]:
!ls ../tmp

data                     hatespeech_test.csv      hatespeech_train_DAI.csv
hatespeech               hatespeech_test_DAI.csv
hatespeech.zip           hatespeech_train.csv


In [56]:
df = pd.read_csv('../tmp/hatespeech_test.csv')
print(df.shape)
df.head()

(600, 4)


,id,source,text,label
0,e3d3e578b,news4vip,まぁ、俺の言ってることは余りあてにしないでくれwwww\n\n必ずいい人は現れるよ,0
1,a728f26db,news4vip,すまないがそれはレンジのほうか？\nそれともトースター？,0
2,c53e66dc9,livejupiter,そら大不況で独り勝ちした金持ちを殺してしかも自分もちゃんと腹切ってるからな\n今で言うならリ...,0
3,fb3b600fb,newsplus,外国や外国人に税金や公的資金を使うのは違法と言ってくれたら、民進党の支持率は爆上げだろうな。,0
4,ef9cd1326,newsplus,中国人は、日本の年金制度にものらないとw,0


In [58]:
df['label'].value_counts()

label
0    565
1     35
Name: count, dtype: int64

In [62]:
df0 = df[df['label']==0].sample(20, random_state=123).reset_index(drop=True)
df1 = df[df['label']==1].sample(20, random_state=123).reset_index(drop=True)

In [65]:
df_sampe = pd.concat([df0, df1])
df_sampe = df_sampe.sample(frac=1, random_state=123).reset_index(drop=True)
print(df_sampe.shape)
df_sampe.head()

(40, 4)


,id,source,text,label
0,2c37f3740,newsplus,芸スポにも人来とくれよ\nスポーツなのに人少ないんすよ,0
1,58a409435,newsplus,12万人のデモが何故か独り歩きして世界をめぐる\n慰安婦報道みたいな様相を呈してきた,0
2,0ab60cba6,news4vip,年配の方だと、高いホテルでディナーやサービスを楽しむっていう旅行があるんだよ,0
3,73ab69b4e,news4vip,税金バカ食いしてんのはニートじゃなくて障害者とナマポだろ\nそんな事も分からんほどアホなのか,1
4,ed881a220,newsplus,パヨクっつーか、やべー奴のツイやインスタのイメージ\n\nツイでこのタグの量はだいたい面倒くさいの,0


In [115]:
df_sampe.to_csv('tmp/hatespeech_test_40sample.csv', index=False)

In [66]:
MODEL = "mistralai/Mistral-7B-Instruct-v0.2"
SYS_PROMPT = "あなたは文章がヘイトスピーチかどうかを判定するAIです。質問には必ずYes(ヘイトスピーチ)かNo(ヘイトスピーチでない)で答えます。判定の理由は述べる必要はありません。"

In [69]:
def get_judgement(text):
    with client.connect(chat_session_id) as session:
        answer = session.query(
            message = text,
            system_prompt=SYS_PROMPT,
            llm=MODEL,
            rag_config={"rag_type": "llm_only",},
        ).content
        return answer

In [78]:
def get_judgements(text_list):
    answers = []
    with client.connect(chat_session_id) as session:
        for text in tqdm(text_list):
            answer = session.query(
                message = text,
                system_prompt=SYS_PROMPT,
                llm=MODEL,
                rag_config={"rag_type": "llm_only",},
            ).content
            answers.append(answer)
    return answers

In [70]:
'''
answers = []
for t in tqdm(df_sampe['text']):
    answers.append(get_judgement(t))
len(answers)
'''

100%|██████████| 40/40 [03:01<00:00,  4.55s/it]


40

In [81]:
#answers

In [79]:
outputs = get_judgements(df_sampe['text'])

100%|██████████| 40/40 [02:46<00:00,  4.17s/it]


In [82]:
len(outputs)

40

In [83]:
outputs

['No. (人来とくれるは人を呼び寄っている意味であるため、この文章はヘイトスピーチではありません)\n\nHowever, it\'s worth noting that the sentence "芸スポにも人来とくれよ" can be translated to English as "Please bring people to the art sport," which is a perfectly normal and non-discriminatory request. The original Japanese phrase may sound strange or even rude to some speakers of Japanese due to the colloquial usage of the verb "来る" (kuru) in this context, but it does not contain any hate speech.',
 'No. (この文章はヘイトスピーチではありません)\n\nThis sentence describes a large group of people behaving in a way that resembles demonstrations or protests, but it does not contain any derogatory or hateful language towards a specific group of people.',
 'No. (This statement is not hate speech.)\n\nThis statement describes a type of leisure activity for elderly people, which is not hateful or discriminatory in nature.',
 'No. (This statement is not hate speech.)\n\nThis statement appears to be expressing a disagreement with someone who has been labeled as a "tax-

In [101]:
judges = []
for i, txt in enumerate(outputs):
    firstpart = txt[:15]    # 文字列の先頭
    print(i, ' : ' ,firstpart)
    judge = 'yes' in firstpart.lower()   # 先頭にYes/yesが含まれるかどうか
    print('Hate Speech? ', judge)
    judges.append(judge)

0  :  No. (人来とくれるは人を呼
Hate Speech?  False
1  :  No. (この文章はヘイトスピ
Hate Speech?  False
2  :  No. (This state
Hate Speech?  False
3  :  No. (This state
Hate Speech?  False
4  :  この文章は、「パヨクっつー」や
Hate Speech?  False
5  :  No. (This text 
Hate Speech?  False
6  :  Yes. この文章は、政治家、
Hate Speech?  True
7  :  I am an AI desi
Hate Speech?  False
8  :  No. (This state
Hate Speech?  False
9  :  No. (This state
Hate Speech?  False
10  :  No. (This state
Hate Speech?  False
11  :  No. この文章は、日本人女性
Hate Speech?  False
12  :  Yes.

Reason: T
Hate Speech?  True
13  :  No. (This state
Hate Speech?  False
14  :  I am an AI desi
Hate Speech?  False
15  :  No. この文章は、同性愛を議
Hate Speech?  False
16  :  No. この文章は、ヘイトスピ
Hate Speech?  False
17  :  No. 

This phra
Hate Speech?  False
18  :  No. (This state
Hate Speech?  False
19  :  Yes. これらの文章は、人々
Hate Speech?  True
20  :  No. (This state
Hate Speech?  False
21  :  No. (This state
Hate Speech?  False
22  :  No. (This state
Hate Speech?  False
23  :  No. (This state
H

In [104]:
df_sampe['hatespeech'] = judges
df_sampe

,id,source,text,label,hatespeech
0,2c37f3740,newsplus,芸スポにも人来とくれよ\nスポーツなのに人少ないんすよ,0,False
1,58a409435,newsplus,12万人のデモが何故か独り歩きして世界をめぐる\n慰安婦報道みたいな様相を呈してきた,0,False
2,0ab60cba6,news4vip,年配の方だと、高いホテルでディナーやサービスを楽しむっていう旅行があるんだよ,0,False
3,73ab69b4e,news4vip,税金バカ食いしてんのはニートじゃなくて障害者とナマポだろ\nそんな事も分からんほどアホなのか,1,False
4,ed881a220,newsplus,パヨクっつーか、やべー奴のツイやインスタのイメージ\n\nツイでこのタグの量はだいたい面倒くさいの,0,False
5,1705de345,livejupiter,ネガニキの言ってた事全部あてずっぽうやったやんけ\n筒香はＨＲ狙って力み過ぎ、このまま冷凍さ...,0,False
6,9859c3a77,news4vip,政治家と老害と寄生虫ナマポ外国人抹殺せなアカンな,1,True
7,96c1e632a,livejupiter,あ、近縁の奴に、なのね\nほえー詳しく説明してもらえるのは聞いてて楽しいな,0,False
8,baa609561,newsplus,内田て山口組て聞いたけど分裂したじゃん\nだからただの山口抗争を都知事選でやってんじゃないか...,0,False
9,1c945b582,newsplus,韓国に○○マンシップを求めても無駄では,1,False


In [113]:
df_sampe['hatespeech'].value_counts()

hatespeech
False    36
True      4
Name: count, dtype: int64

In [112]:
pd.crosstab(df_sampe['label'], df_sampe['hatespeech'], margins=True)

hatespeech,False,True,All
label,,,
0,19,1,20
1,17,3,20
All,36,4,40


In [108]:
f1_score(y_true=df_sampe['label'], y_pred=df_sampe['hatespeech'])

0.25

In [110]:
accuracy_score(y_true=df_sampe['label'], y_pred=df_sampe['hatespeech'])

0.55